In [1]:
def reduce_quiet(y):
    mx_bin = 0b100011011
    max_width = len(bin(y))
    while y > 0b11111111:
        curr_len = len(bin(y))
        temp_mx = bin(mx_bin).ljust(curr_len,'0')
        temp_mx = int(temp_mx,2)
        y = y ^ temp_mx
    return y

In [2]:
def reduce(y):
    mx_bin = 0b100011011
    max_width = len(bin(y))
    while y > 0b11111111:
        print(bin(y)[2:].rjust(max_width,' '))
        curr_len = len(bin(y))
        temp_mx = bin(mx_bin).ljust(curr_len,'0')
        temp_mx = int(temp_mx,2)
        y = y ^ temp_mx
        print(bin(temp_mx)[2:].rjust(max_width,' '))
        print('-'.rjust(max_width,'-'))
    print(bin(y)[2:].rjust(max_width,' '))
    return y

In [72]:
def gf_div(n,d):
    #print("----------DIV "+hex(n)+" and "+hex(d)+"----------")

    max_width = len(bin(n))
    min_width = len(bin(d))
    temp_n = n
    temp_d = d
    q = 0
    r = 0
    #print(bin(temp_n).rjust(max_width,' '))

    while len(bin(temp_n)) >= min_width:
        curr_len = len(bin(temp_n))
        m = (curr_len - min_width)
        m = 2**m
        q = q ^ m
        temp_d = bin(d).ljust(curr_len,'0')
    #    print(temp_d.rjust(max_width,' '))
        temp_d = int(temp_d,2)
        temp_n = temp_n ^ temp_d
        r = temp_n
    #    print('-'.rjust(max_width,'-'))
    #    print(bin(r).rjust(max_width,' '))
    #print("Q = "+hex(q)+ ", R = "+hex(r))    
    #print("---------------END--------------------")
    return [q,r]

In [73]:
def gf_mul(a,b):
    #print("----------MUL "+hex(a)+" and "+hex(b)+"----------")
    a = bin(a)[2:]
    a = a[::-1]
    prod =  0
    max_width = len(bin(b))
    for i in range(len(a)):
        if a[i] == '1':
            temp = (b<<i)
            if(temp>0b11111111):
                temp = reduce_quiet(temp)                
    #        print(bin(temp).rjust(max_width,' '))
            prod = prod ^ temp
    #print("-".rjust(max_width,'-'))
    #print(bin(prod).rjust(max_width,' '))
    #print("---------------END--------------------")
    #print("--------------------------------------")
    return prod

In [74]:
def gf_eea(N,a):

    R=[N,a,0,0,0,0,0,0,0,0,0,0,0,0,0]
    S=[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    T=[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
    Q=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    count = 2
    while R[count-1] > 1:
        #print("determining r and q: "+hex(R[count-2])+" / "+ hex(R[count-1]))
        temp = gf_div(R[count-2],R[count-1])
        q_temp = temp[0]
        r_temp = temp[1]
        R[count] = r_temp
        Q[count] = q_temp
        S_temp = (S[count-2] ^ gf_mul(S[count-1],q_temp))
        T_temp=(T[count-2] ^ gf_mul(T[count-1],q_temp))
        T[count] = T_temp
        S[count] = S_temp
        count=count+1
    print ("Find inverse of "+hex(a) + " Using EEA")
    print ('  #' +" | "+ "  Q   "+" | "+"  R   "+" | "+"  S   "+" | "+"   T ")
    print("------------------------------------------")
    for x in range(count):
            print(str(x).rjust(3,' ') +" | "+ hex(Q[x]).ljust(6,' ')+" | "+hex(R[x]).rjust(6,' ')+" | "+hex(S[x]).rjust(6,' ')+" | "+hex(T[x]).rjust(6,' '))
    
    return T[count-1]

In [75]:
def decrypt_block(cipher_block):
    a = 0x3F
    b = 0xAF
    M = (reduce(gf_mul(a,(cipher_block ^ b) )  ))
    return M

In [76]:
def encrypt_block(plain_block):
    a = 0x19
    b = 0xAF
    C = reduce((reduce(gf_mul(a,plain_block)) ^ b ))
    return C

In [77]:
def encrypt_text(plain_text):
    temp = ''
    for block in plain_text:
        temp = temp +" "+ hex(encrypt_block(block))
    print(temp)

In [78]:
def decrypt_text(cipher_text):
    temp = ''
    for block in cipher_text:
        temp = temp +" "+ hex(decrypt_block(block))
    print(temp)

In [79]:
plain_text = (0x4d,0x61,0x74,0x68)
encrypt_text(plain_text)

  10101111
  0
  1110
  10100001
  11111000
  1010111
  11011111
  1110000
 0x0 0xa1 0x57 0x70


In [80]:

hex(gf_eea(0x11b,0x19))

Find inverse of 0x19 Using EEA
  # |   Q    |   R    |   S    |    T 
------------------------------------------
  0 | 0x0    |  0x11b |    0x1 |    0x0
  1 | 0x0    |   0x19 |    0x0 |    0x1
  2 | 0x1f   |    0xc |    0x1 |   0x1f
  3 | 0x2    |    0x1 |    0x2 |   0x3f


'0x3f'

In [87]:
#Question 1.c
cipher_text = (0x0,0xa1,0x57,0x70)
decrypt_text(cipher_text)

  1001101
  1100001
  1110100
  1101000
 0x4d 0x61 0x74 0x68
